<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/40_3_%3D%3D_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Дообучи русскоязычную модель GPT для генерации фейковых новостей.
2. Датасет необходимо найти самостоятельно.
3. Приведи 5-10 примеров сгенерированных фейковых новостей на русском языке.
4. Подбери параметры генерации, улучшите качество новостей.
5. Изучи model.generate более детально.
6. Добейся такой генерации, чтобы не было обрыва мысли, новость заканчивалась либо точкой, либо восклицательным знаком, либо вопросительным (если уместно).



In [ ]:
# Импортируем необходимые библиотеки
from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [ ]:
# Определяем GPT модель и токенизатор
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(DEVICE)

In [ ]:
# Загружаем и создаем датасет
train_path = 'train_dataset.txt'
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=64)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# Параметры для обучения
training_args = TrainingArguments(
    output_dir="./finetuned",       # директория с выходными данными
    overwrite_output_dir=True,      # перезаписываем содержимое выходной директории при каждом запуске
    num_train_epochs=200,           # число эпох обучения
    per_device_train_batch_size=32, # batch size для обучения
    per_device_eval_batch_size=32,  # batch size для выполнения
    warmup_steps=10,                # количество шагов для "прогрева" (управление скоростью обучения)
    gradient_accumulation_steps=16,) # накопление градиента (16 шагов накапливаем градиенты для batch_size, эмуляция вычисления на пакете 16 * 32 для слабых GPU)

# Инициализируем класс обучения
trainer = Trainer(
    model=model,                 # модель
    args=training_args,          # параметры обучения
    data_collator=data_collator, # загрузчик данных
    train_dataset=train_dataset, # датасет для обучения
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)) # оптимизатор

In [ ]:
# Запуск обучения
trainer.train()

In [ ]:

text = "Почему GPT так жестока?\nМожет просто она - человек?"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
print()